<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/%EA%B3%BC%EC%A0%9C/json_to_text%2C_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 샘플파일
!gdown 1rswWV67Idv8OXI1qQ7o5aSSWvP_uA5MW
# 맑음고딕 폰트
!gdown 1G4jAcJLy_1PC9YQR36t7jMapj9da4vPd

Downloading...
From: https://drive.google.com/uc?id=1rswWV67Idv8OXI1qQ7o5aSSWvP_uA5MW
To: /content/sample.zip
100% 2.95M/2.95M [00:00<00:00, 168MB/s]
Downloading...
From: https://drive.google.com/uc?id=1G4jAcJLy_1PC9YQR36t7jMapj9da4vPd
To: /content/malgun.ttf
100% 13.5M/13.5M [00:00<00:00, 34.6MB/s]


In [2]:
import zipfile
import os
import json
from PIL import Image, ImageDraw, ImageFont

In [3]:
!mkdir /content/sample
%cd /content/sample

/content/sample


In [ ]:
!unzip /content/sample.zip

In [5]:
%cd /content/
!mkdir /content/text
!mkdir /content/json
!mkdir /content/jpg

/content


In [15]:
foilderlist = os.listdir('/content/sample')

for i in foilderlist:
    per_path = f'/content/sample/{i}/20per'
    sent_path = f'/content/sample/{i}/2~3sent'
    per_list = os.listdir(per_path)
    sent_list = os.listdir(sent_path)

    for j in per_list:
        with open(f'{per_path}/{j}', 'r', encoding='utf-8') as file:
            text = json.load(file)

            image_path0 = f'{j[:-5]}.jpg'
            ground_truth_parse = f" {text['Meta(Refine)']['passage']}"

            # Create a dictionary with the desired format
            sample = {
                "file_name": image_path0,
                "ground_truth": {
                    "gt_parses": [
                        {
                            "text_sequence": ground_truth_parse
                        }
                    ]
                }
            }

            # Save the dictionary as a JSON file
            with open(f'/content/json/{j[:-5]}.json', 'w', encoding='utf-8') as fi:
                json.dump(sample, fi, ensure_ascii=False, indent=4)

In [7]:
foilderlist = os.listdir('/content/sample')

for i in foilderlist:
    per_path = f'/content/sample/{i}/20per'
    sent_path = f'/content/sample/{i}/2~3sent'
    per_list = os.listdir(per_path)
    sent_list = os.listdir(sent_path)

    for j in per_list:
        with open(f'{per_path}/{j}', 'r', encoding='utf-8') as file:
            text = json.load(file)

            image_path0 = f'{j[:-5]}.jpg'
            ground_truth_parse = f" {text['Meta(Refine)']['passage']}"

            # Create a dictionary with the desired format
            sample = {
                "file_name": image_path0,
                "ground_truth": {
                    "gt_parses": [
                        {
                            "text_sequence": ground_truth_parse
                        }
                    ]
                }
            }

            # Save the dictionary as a JSON file
            with open(f'/content/json/{j[:-5]}.json', 'w', encoding='utf-8') as fi:
                json.dump(sample, fi, ensure_ascii=False, indent=4)

        # Save the extracted passage to the 'test' directory
            with open(f'/content/text/{j[:-5]}.txt', 'w', encoding='utf-8') as f:
                f.write(f'{ground_truth_parse}')

        # Create an image with the text
        image_width = 800
        image_height = 1200
        font_size = 18 # font size
        left_margin = 50 # 왼쪽 여백
        right_margin = 50 # 오른쪽 여백
        vertical_space = 10 # 문단 여백
        top_margin = 50 # 맨 위 여백
        bottom_margin = 50 # 맨 아래 여백

        font_path = "/content/malgun.ttf"  # Replace with your font path

        # Load a Unicode font
        font = ImageFont.truetype(font_path, font_size)

        # Calculate image height and initialize variables
        lines = []
        line = ""

        for word in ground_truth_parse.split():
            # Check if adding the current word exceeds the available width
            if font.getsize(line + " " + word)[0] <= (image_width - left_margin - right_margin):
                line += " " + word
            else:
                lines.append(line)
                line = word

        lines.append(line)  # Append the last line

        # Calculate image height based on the number of lines
        line_height = font.getsize('A')[1]  # Height of a line of text
        image_height = line_height * len(lines) + (len(lines) - 1) * vertical_space + top_margin + bottom_margin

        # Create a new image with margins
        image = Image.new('RGB', (image_width, image_height), color='white')
        draw = ImageDraw.Draw(image)

        # Draw the text on the image with margins
        y_position = top_margin  # Adjust the starting y position
        text_color = (0, 0, 0)  # Black color

        for line in lines:
            x_position = left_margin  # Adjust the starting x position
            draw.text((x_position, y_position), line, fill=text_color, font=font)
            y_position += line_height + vertical_space

        # Save the image as JPG
        image.save(f'/content/jpg/{j[:-5]}.jpg')

<ipython-input-7-17ccff53c8a8>:54: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  if font.getsize(line + " " + word)[0] <= (image_width - left_margin - right_margin):
<ipython-input-7-17ccff53c8a8>:63: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  line_height = font.getsize('A')[1]  # Height of a line of text


In [16]:
import os
import shutil
from sklearn.model_selection import train_test_split

# 데이터가 들어있는 폴더 경로
json_folder = '/content/json'
jpg_folder = '/content/jpg'

# 데이터를 저장할 분할된 폴더 경로
train_folder = '/content/dataset/train'
test_folder = '/content/dataset/test'
valid_folder = '/content/dataset/valid'

# JSON 파일과 이미지 파일 목록을 가져옴
json_files = [f for f in os.listdir(json_folder) if f.endswith('.json')]
jpg_files = [f for f in os.listdir(jpg_folder) if f.endswith('.jpg')]

# 데이터를 train/test/valid로 분할
train_json, test_json, train_jpg, test_jpg = train_test_split(json_files, jpg_files, test_size=0.2, random_state=42)
train_json, valid_json, train_jpg, valid_jpg = train_test_split(train_json, train_jpg, test_size=0.2, random_state=42)

# 각 분할에 해당하는 폴더 생성
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)
os.makedirs(valid_folder, exist_ok=True)

# 파일을 해당 폴더로 복사
for json_file, jpg_file in zip(train_json, train_jpg):
    shutil.copy(os.path.join(json_folder, json_file), os.path.join(train_folder, json_file))
    shutil.copy(os.path.join(jpg_folder, jpg_file), os.path.join(train_folder, jpg_file))

for json_file, jpg_file in zip(test_json, test_jpg):
    shutil.copy(os.path.join(json_folder, json_file), os.path.join(test_folder, json_file))
    shutil.copy(os.path.join(jpg_folder, jpg_file), os.path.join(test_folder, jpg_file))

for json_file, jpg_file in zip(valid_json, valid_jpg):
    shutil.copy(os.path.join(json_folder, json_file), os.path.join(valid_folder, json_file))
    shutil.copy(os.path.join(jpg_folder, jpg_file), os.path.join(valid_folder, jpg_file))


### 데이터 변환

In [19]:
!pip install synthtiger

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.0/127.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for blend-modes: filename=blend_modes-2.1.0-py3-none-any.whl size=9539 sha256=4e5b30b8ccb9bc6199c3b0105e875ca9228c1aa4fb98a9c8e541f7dcbeaf5b85
  Stored in directory: /root/.cache/pip/wheels/68/59/0f/7f3f9e6e1fcbbaf41376ce408a4d1f14df7546bad2ced90fd4
  Created wheel for pytweening: filename=pytweening-1.0.7-py3-none-any.whl size=6198 sha256=82ce75b1a5e3f78f67f4ca150b50cb0e67cdca16615f3d5b4f4a4a4a12f5b83f
  Stored in directory: /root/.cache/pip/wheels/ae/a3/d7/36c45539416215425b3247e37b691a98a03b1db7b13b7f9632
Successfully built blend-modes pytweening


In [23]:
%cd /content
!git clone https://github.com/clovaai/donut/tree/master/synthdog.git


/content
Cloning into 'synthdog'...
fatal: repository 'https://github.com/clovaai/donut/tree/master/synthdog.git/' not found


### dount 실험

In [ ]:
%cd /content
!git clone https://github.com/clovaai/donut.git
%cd donut

In [10]:
!pip install transformers==4.25.1
!pip install pytorch-lightning==1.6.4
!pip install timm==0.5.4
!pip install gradio
!pip install donut-python

  Attempting uninstall: markdown-it-py
    Found existing installation: markdown-it-py 3.0.0
    Uninstalling markdown-it-py-3.0.0:
      Successfully uninstalled markdown-it-py-3.0.0
  Attempting uninstall: mdit-py-plugins
    Found existing installation: mdit-py-plugins 0.4.0
    Uninstalling mdit-py-plugins-0.4.0:
      Successfully uninstalled mdit-py-plugins-0.4.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 28.4 MB/s eta 0:00:00
  Created wheel for zs

#### 이미지 넣고 실험

In [ ]:
import torch
print ( "CUDA 사용 가능:" , torch.cuda.is_available())
!nvcc --version
# 이제 관련 파일과 라이브러리를 다운로드할 수 있습니다. 다음 코드 줄은 donut 라이브러리를 포함하여 모든 종속성을 설치해야 합니다
#pip install donut-python.

!git clone https://github.com/clovaai/donut.git
!cd donut && pip install .
# Making an inference using the CORD fine-tuned model
# First, we’ll demonstrate basic usage of the model.

In [37]:
import torch
from donut import DonutModel
from PIL import Image

model = DonutModel.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")
if torch.cuda.is_available():
    model.half()
    device = torch.device("cuda")
    model.to(device)
else:
    model.encoder.to(torch.bfloat16)
model.eval()
image = Image.open("/content/jpg/REPORT-briefing-20091-00001.jpg")
image.convert("RGB")
output = model.inference(image=image, prompt="<s_cord-v2>")
output

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of the model checkpoint at naver-clova-ix/donut-base-finetuned-cord-v2 were not used when initializing DonutModel: ['encoder.model.norm.weight', 'encoder.model.norm.bias']
- This IS expected if you are initializing DonutModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DonutModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'predictions': [{'cnt': '5'}]}

#### 데모버전

In [20]:
import argparse
import gradio as gr
import torch
from PIL import Image

from donut import DonutModel

In [16]:
def demo_process_vqa(input_img, question):
    global pretrained_model, task_prompt, task_name
    input_img = Image.fromarray(input_img)
    user_prompt = task_prompt.replace("{user_input}", question)
    output = pretrained_model.inference(input_img, prompt=user_prompt)["predictions"][0]
    return output


def demo_process(input_img):
    global pretrained_model, task_prompt, task_name
    input_img = Image.fromarray(input_img)
    output = pretrained_model.inference(image=input_img, prompt=task_prompt)["predictions"][0]
    return output

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--task", type=str, default="cord-v2")
parser.add_argument("--pretrained_path", type=str, default="naver-clova-ix/donut-base-finetuned-cord-v2")
args, left_argv = parser.parse_known_args()

task_name = args.task
if "docvqa" == task_name:
    task_prompt = "<s_docvqa><s_question>{user_input}</s_question><s_answer>"
else:  # rvlcdip, cord, ...
    task_prompt = f"<s_{task_name}>"

pretrained_model = DonutModel.from_pretrained(args.pretrained_path)

if torch.cuda.is_available():
    pretrained_model.half()
    device = torch.device("cuda")
    pretrained_model.to(device)
else:
    pretrained_model.encoder.to(torch.bfloat16)

pretrained_model.eval()

demo = gr.Interface(
    fn=demo_process_vqa if task_name == "docvqa" else demo_process,
    inputs=["image", "text"] if task_name == "docvqa" else "image",
    outputs="json",
    title=f"Donut 🍩 demonstration for `{task_name}` task",
)


demo.launch(debug=False, share=True`)

Some weights of the model checkpoint at naver-clova-ix/donut-base-finetuned-cord-v2 were not used when initializing DonutModel: ['encoder.model.norm.weight', 'encoder.model.norm.bias']
- This IS expected if you are initializing DonutModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DonutModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d8fbcfc1376251dc7c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


### 훈련 .. 실패중

In [23]:
config_data ={
  "_name_or_path": "naver-clova-ix/donut-base",
  "align_long_axis": False,
  "architectures": [
    "DonutModel"
  ],
  "decoder_layer": 4,
  "encoder_layer": [
    2,
    2,
    14,
    2
  ],
  "input_size": [
    800,
    1200
  ],
  "max_length": 768,
  "max_position_embeddings": 768,
  "model_type": "donut",
  "torch_dtype": "float32",
  "transformers_version": "4.11.3",
  "window_size": 10
}

with open('/content/donut/config.json', 'w', encoding = 'utf-8') as file:
    json.dump(config_data, file, indent=4)

In [18]:
model = DonutModel.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of the model checkpoint at naver-clova-ix/donut-base-finetuned-cord-v2 were not used when initializing DonutModel: ['encoder.model.norm.bias', 'encoder.model.norm.weight']
- This IS expected if you are initializing DonutModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DonutModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
# train the model
!python /content/donut/train.py --config /content/donut/config/train_cord.yaml --pretrained_model_name_or_path  "naver-clova-ix/donut-base-finetuned-cord-v2"

2023-08-21 08:51:38.925612: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-21 08:51:40.065833: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
resume_from_checkpoint_path: None
result_path: ./result
pretrained_model_name_or_path: naver-clova-ix/donut-base-finetuned-cord-v2
dataset_name_or_paths: /content/dataset
sort_json_key: False
train_batch_sizes: 
  - 1
val_batch_sizes: 
  - 1
input_size: 
  - 1600
  - 1920
max_length: 700
align_long_axis: False
num_nodes: 1
seed: 2022
lr: 0.0003
warmup_steps: 300
num_training_samples_per_epoch: 800
max_epochs: 30
max_steps: -1
num_workers: 8
val_check_interval: 1.0
check_val_every_n_epoch: 10
gradient_clip_val: 1.0
verbose: True
exp_name: train_cord
e

In [99]:
import json
from datasets import load_dataset


dataset = load_dataset("/content/Dataset", split="train")
for sample in dataset:
    print("osh",ground_truth)
    print("gt_parses" in ground_truth)
    if "gt_parses" in ground_truth:  # when multiple ground truths are available, e.g., docvqa
        assert isinstance(ground_truth["gt_parses"], list)
        gt_jsons = ground_truth["gt_parses"]
    else:
        print(ground_truth["gt_parse"])
        print(type(ground_truth["gt_parse"]))
        assert "gt_parse" in ground_truth and isinstance(ground_truth["gt_parse"], dict)
        gt_jsons = [ground_truth["gt_parse"]]


Resolving data files:   0%|          | 0/2000 [00:00<?, ?it/s]

osh {'gt_parse': ' 이러한 유행 특성 때문에 의료체계는 무증상 감염, 무증상 경증환자 치료부분 외에 중환자 치료 등은 충분한 여유를 가지고 있습니다. 중환자실은 전체의 73%가 비어져 있어 588병상이 즉시 가용 가능합니다. 중등도 환자를 위한 감염병 전담병원도 53% 비어져 있어 3,990병상이 가용 가능합니다. 다만, 수도권의 무증상 경증 환자 증가로 생활치료센터 입실은 점차 증가하고 있습니다. 생활치료센터는 현재 29%, 2,327병상의 여유가 있습니다. 아직 하루 이상 병상대기를 하는 환자는 1명도 없으나, 무증상 경증 환자는 계속 많이 발생할 것으로 예상하고 있습니다. 정부는 지자체와 함께 생활치료센터를 신속하게 확충하는 중이며, 전국적으로 19개, 약 5,000여 병상의 생활치료센터 시설 개소를 준비하고 있습니다. 정부는 방역지표를 예의주시하며 방역과 의료 대응에 최선을 다하겠습니다. 오늘 중대본에서는 수도권의 사회적 거리두기 단계 격상에 따른 후속조치에 관해 논의하였습니다. 먼저, 중소기업벤처부는 지난 7월 7일 공표되고 10월 8일부터 시행되는 소상공인지원법 개정안에 따른 손실보상을 준비하고 있습니다. 이번 개정안에 따르면, 7월 7일 이후에 감염병예방법에 따른 집합금지 또는 운영시간 제한 등으로 인해 심각한 경영상 손실이 발생할 소상공인은 손실보상이 가능합니다. 보상금의 지급대상과 금액 등 구체적인 기준은 손실보상심의위원회의 심의를 거쳐 중기부 장관이 고시할 예정입니다.'}
False
 이러한 유행 특성 때문에 의료체계는 무증상 감염, 무증상 경증환자 치료부분 외에 중환자 치료 등은 충분한 여유를 가지고 있습니다. 중환자실은 전체의 73%가 비어져 있어 588병상이 즉시 가용 가능합니다. 중등도 환자를 위한 감염병 전담병원도 53% 비어져 있어 3,990병상이 가용 가능합니다. 다만, 수도권의 무증상 경증 환자 증가로 생활치료센터 입실은 점차 증가하고 있습니다. 생활치료센터는 현재 29%, 2,327병상의 여유가 있습니다. 아직 

AssertionError: ignored